In [1]:
import pandas as pd
# from bs4 import BeautifulSoup
import emoji
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import sklearn
import numpy as np
import random
import os

# Возьмем очищенный и обработанный сэмпл данных с прошлого дз

In [2]:
sample_data = pd.read_csv('sampled_clean_text.csv')

### на всякий случай, чтобы пролезло в github я заархивировал, но я использовал без архивирования

In [3]:
sample_data.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
sample_data.head()

,index,text,title,topic
0,149397,ученый новосибирский институт химический биоло...,сибирский ученый открыть новый способ выявлени...,69-я параллель
1,142717,норильский никель планировать год инвестироват...,норникель потратить инвестиция миллиард доллар...,69-я параллель
2,50131,министерство труд предложить дарить мать набор...,минтруд собраться раздать новорожденный чепчик...,69-я параллель
3,139634,полуостров ямал турист разработать маршрут пос...,турист ямал предложить посетить самый восточны...,69-я параллель
4,105236,миллиард рубль инвестировать строительство нов...,создание мурманский транспортный узел вложить ...,69-я параллель


Зафиксируем все сиды 

In [5]:
RANDOM_STATE = 42

def set_seed(seed: int = RANDOM_STATE) -> None:
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(RANDOM_STATE)

Random seed set as 42


Посмотрим, все ли данные являются текстом в выборке, если нет, то удалим этот объект

In [6]:
for tt in sample_data['text'].tolist():
    if not isinstance(tt, str):
        print(tt, type(tt))

nan <class 'float'>


In [7]:
sample_data = sample_data.dropna(subset=['text'])

In [8]:
for tt in sample_data['text'].tolist():
    if isinstance(tt, float):
        print(tt)

# Разделим наш датасет на train, test, valid

In [9]:
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(sample_data, test_size=0.4, stratify=sample_data["topic"], random_state=RANDOM_STATE)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df["topic"], random_state=RANDOM_STATE)

In [10]:
train_df.shape,  test_df.shape , val_df.shape

((65910, 4), (21970, 4), (21970, 4))

# Обучим word2vec-эмбеддинги с помощью библиотеки gensim

In [11]:
import gensim.downloader as api
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import gensim.models
from gensim.models import Word2Vec

In [12]:
tokenized_texts = [tt.split() for tt in train_df['text'].tolist()]

In [13]:
model = gensim.models.Word2Vec(
    sentences=tokenized_texts,  # Обучение запускается здесь
    vector_size=100,
    window=5,
    min_count=5,
    sg=1,
    negative=5,
    epochs=10,  
    seed=RANDOM_STATE,
)

Визуально оценим качество получившихся эмбедингов

In [14]:
# И проверим ее внутренее качество
print(model.wv.doesnt_match(["Вице-премьер", "президент", "комиссар"]))
print(model.wv.most_similar("культура", topn=5))

комиссар
[('культурный', 0.7291277050971985), ('наука', 0.7167322039604187), ('культурноисторический', 0.7088230848312378), ('виноделие', 0.6798076033592224), ('мироненко', 0.6734700798988342)]


# Загрузим предобученные эмбеддинги из navec и rusvectores

In [15]:
!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_news_v1_1B_250K_300d_100q.tar

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [16]:
from navec import Navec
import numpy as np

navec_model = Navec.load("navec_news_v1_1B_250K_300d_100q.tar")

def text_to_avg_navec(text, model):
    vectors = [model.get(word) for word in text.split() if word in model]
    return np.mean(vectors, axis=0) if len(vectors) > 0 else np.zeros(300) # 300 исходя из размерности эмбедингов которая указана в названии файла

X_train_navec = np.array([text_to_avg_navec(text, navec_model) for text in train_df['text']])
X_val_navec = np.array([text_to_avg_navec(text, navec_model) for text in val_df['text']])
X_test_navec = np.array([text_to_avg_navec(text, navec_model) for text in test_df['text']])

In [17]:
X_train_navec

array([[-0.19668593, -0.07471703, -0.13901667, ..., -0.062306  ,
         0.09618099,  0.00421107],
       [ 0.00278849, -0.07244179,  0.07407308, ...,  0.02509489,
         0.00299011, -0.10284942],
       [-0.06943322, -0.05035128, -0.05246846, ..., -0.03841982,
        -0.04100087, -0.08578858],
       ...,
       [-0.01949179, -0.29004455, -0.03907801, ..., -0.09301674,
        -0.03565096, -0.10774067],
       [ 0.07339207, -0.13412614,  0.00996305, ...,  0.01967051,
         0.08979401, -0.04948989],
       [ 0.06798457, -0.14860465, -0.02356817, ...,  0.06375785,
         0.14657015, -0.04427757]])

In [18]:
from gensim.models import KeyedVectors
# разархивированный bin файл модели 
model_path = "model.bin"

rusvectores_model = KeyedVectors.load_word2vec_format(
    model_path, 
    binary=True, 
    unicode_errors='ignore' 
)

In [19]:
def text_to_avg_rusvectores(text, model):
    vectors = [model[word] for word in text.split() if word in model]
    return np.mean(vectors, axis=0) if len(vectors) > 0 else np.zeros(model.vector_size)

X_train_rv = np.array([text_to_avg_rusvectores(text, rusvectores_model) for text in train_df['text']])
X_val_rv = np.array([text_to_avg_rusvectores(text, rusvectores_model) for text in val_df['text']])
X_test_rv = np.array([text_to_avg_rusvectores(text, rusvectores_model) for text in test_df['text']])

In [20]:
rusvectores_model.most_similar(positive=['президент_NOUN'], topn=10)

[('президент_PROPN', 0.5634657144546509),
 ('россия::владимир::путин_PROPN', 0.4750259518623352),
 ('путин_PROPN', 0.45360875129699707),
 ('экс-президент_NOUN', 0.44043391942977905),
 ('президентский_ADJ', 0.4396216869354248),
 ('владимир::путин_PROPN', 0.43582114577293396),
 ('и.о.президент_NOUN', 0.43094614148139954),
 ('глава_NOUN', 0.42920252680778503),
 ('премьер-министр_NOUN', 0.42138782143592834),
 ('барак_PROPN', 0.42040306329727173)]

# Обучим sklearn.linear_model.LogisticRegression с тремя вариантами векторизации текстов и сравните их качество между собой на валидационной выборке

In [21]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import numpy as np

In [22]:
def text_to_vector(text, model):
    words = text.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

In [24]:
X_train_W2V = np.array([text_to_vector(text, model) for text in train_df['text']])
X_val_W2V = np.array([text_to_vector(text, model) for text in val_df['text']])
X_test_W2V = np.array([text_to_vector(text, model) for text in test_df['text']])

### Посмотрим качество на валидационной выборке для 3х моделей

результаты W2V: 

In [25]:
logreg = LogisticRegression().fit(X_train_W2V, train_df['topic'])
preds = logreg.predict(X_val_W2V)
print(classification_report(val_df['topic'], preds))

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                   precision    recall  f1-score   support

   69-я параллель       0.67      0.05      0.10        38
           Бизнес       0.46      0.10      0.17       216
      Бывший СССР       0.78      0.72      0.75      1557
              Дом       0.81      0.69      0.75       646
         Из жизни       0.62      0.54      0.58       827
   Интернет и СМИ       0.72      0.64      0.68      1327
             Крым       0.00      0.00      0.00        20
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.84      0.86      0.85      1605
              Мир       0.76      0.83      0.80      4079
  Наука и техника       0.82      0.80      0.81      1579
      Путешествия       0.69      0.48      0.56       192
           Россия       0.72      0.81      0.76      4765
Силовые структуры       0.46      0.18      0.26       584
            Спорт       0.96      0.96      0.96      1927
         Ценности       0.83      0.73      0.78       

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


результаты NAVEC: 

In [26]:
logregNavec = LogisticRegression().fit(X_train_navec, train_df['topic'])
preds = logregNavec.predict(X_val_navec)
print(classification_report(val_df['topic'], preds))

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                   precision    recall  f1-score   support

   69-я параллель       0.54      0.18      0.27        38
           Бизнес       0.49      0.13      0.21       216
      Бывший СССР       0.78      0.75      0.76      1557
              Дом       0.83      0.72      0.77       646
         Из жизни       0.59      0.54      0.56       827
   Интернет и СМИ       0.73      0.66      0.69      1327
             Крым       1.00      0.05      0.10        20
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.83      0.87      0.85      1605
              Мир       0.77      0.82      0.79      4079
  Наука и техника       0.81      0.80      0.81      1579
      Путешествия       0.65      0.46      0.54       192
           Россия       0.73      0.81      0.77      4765
Силовые структуры       0.47      0.21      0.29       584
            Спорт       0.96      0.97      0.96      1927
         Ценности       0.81      0.73      0.77       

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


результаты rusvectors: 

In [27]:
logregRus = LogisticRegression().fit(X_train_rv, train_df['topic'])
preds = logregRus.predict(X_val_rv)
print(classification_report(val_df['topic'], preds))

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                   precision    recall  f1-score   support

   69-я параллель       0.00      0.00      0.00        38
           Бизнес       0.00      0.00      0.00       216
      Бывший СССР       0.00      0.00      0.00      1557
              Дом       0.00      0.00      0.00       646
         Из жизни       0.00      0.00      0.00       827
   Интернет и СМИ       0.00      0.00      0.00      1327
             Крым       0.00      0.00      0.00        20
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.00      0.00      0.00      1605
              Мир       0.00      0.00      0.00      4079
  Наука и техника       0.00      0.00      0.00      1579
      Путешествия       0.00      0.00      0.00       192
           Россия       0.22      1.00      0.36      4765
Силовые структуры       0.00      0.00      0.00       584
            Спорт       0.00      0.00      0.00      1927
         Ценности       0.00      0.00      0.00       

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Попробуем улучшить качество модели, взяв для ее обучения W2V набор эмбеддингов и используя его с взвешиванием через tf-idf

In [28]:
from joblib import Parallel, delayed
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

In [29]:
def text_to_vector_tfidf(text, model, idf_dict):
    words = text.split()
    known_words = [word for word in words if word in model.wv]
    
    if not known_words:
        return np.zeros(model.vector_size)
    
    word_vectors = []
    for word in known_words:
        try:
            word_vectors.append(model.wv[word] * idf_dict[word])
        except KeyError: 
            continue
    
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)

    return np.mean(word_vectors, axis=0)

Для обработки по батчам

In [30]:
def process_batch(batch, model, idf_dict):
    return np.array([text_to_vector_tfidf(text, model, idf_dict) for text in batch])

In [31]:
def vectorize_texts_in_batches(texts, model, idf_dict, batch_size=1000, n_jobs=2):
    batches = [texts[i:i+batch_size] for i in range(0, len(texts), batch_size)]
    
    results = []
    with tqdm(total=len(batches), desc="Vectorizing texts", unit="batch") as pbar:
        for batch_result in Parallel(n_jobs=n_jobs)(
            delayed(process_batch)(batch, model, idf_dict) for batch in batches
        ):
            results.append(batch_result)
            pbar.update(1) 

    return np.vstack(results)

Обучим tf-idf:

In [32]:
vectorizer = TfidfVectorizer()
vectorizer.fit(train_df['text'])  

TfidfVectorizer()

Теперь с коэфициентами TF-IDF

In [33]:
feature_names = vectorizer.get_feature_names_out() 
idf_dict = dict(zip(feature_names, vectorizer.idf_))

In [34]:
from tqdm import tqdm
X_train_w2v_tfidf = vectorize_texts_in_batches(train_df['text'], model, idf_dict, batch_size=1000, n_jobs=2)

Vectorizing texts: 100%|████████████████████████████████████████████████████████████| 66/66 [00:47<00:00,  1.40batch/s]


In [35]:
X_val_w2v_tfidf = vectorize_texts_in_batches(val_df['text'], model, idf_dict, batch_size=1000, n_jobs=2)

Vectorizing texts: 100%|████████████████████████████████████████████████████████████| 22/22 [00:16<00:00,  1.34batch/s]


In [36]:
X_test_w2v_tfidf = vectorize_texts_in_batches(test_df['text'], model, idf_dict, batch_size=1000, n_jobs=2)

Vectorizing texts: 100%|████████████████████████████████████████████████████████████| 22/22 [00:15<00:00,  1.41batch/s]


Проверим качество нашей комбинированной модели на валидационной выборке

In [37]:
LR_w2v_tfidf= LogisticRegression(max_iter=1000)
LR_w2v_tfidf.fit(X_train_w2v_tfidf, train_df['topic'])

preds = LR_w2v_tfidf.predict(X_val_w2v_tfidf)
print(classification_report(val_df['topic'], preds))

                   precision    recall  f1-score   support

   69-я параллель       0.33      0.13      0.19        38
           Бизнес       0.48      0.15      0.23       216
      Бывший СССР       0.78      0.73      0.75      1557
              Дом       0.81      0.71      0.75       646
         Из жизни       0.63      0.55      0.59       827
   Интернет и СМИ       0.71      0.65      0.68      1327
             Крым       0.29      0.10      0.15        20
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.85      0.86      0.86      1605
              Мир       0.77      0.82      0.79      4079
  Наука и техника       0.83      0.80      0.81      1579
      Путешествия       0.66      0.51      0.57       192
           Россия       0.72      0.80      0.76      4765
Силовые структуры       0.44      0.18      0.26       584
            Спорт       0.96      0.96      0.96      1927
         Ценности       0.82      0.78      0.80       

# Финально сравнение качества всех моделей на тестовой выборке

In [38]:
preds = LR_w2v_tfidf.predict(X_test_w2v_tfidf)
print(classification_report(test_df['topic'], preds))

                   precision    recall  f1-score   support

   69-я параллель       0.50      0.18      0.27        38
           Бизнес       0.38      0.12      0.18       216
      Бывший СССР       0.78      0.76      0.77      1557
              Дом       0.82      0.74      0.78       646
         Из жизни       0.63      0.52      0.57       828
   Интернет и СМИ       0.71      0.65      0.68      1326
             Крым       0.17      0.05      0.08        20
    Культпросвет        0.33      0.10      0.15        10
         Культура       0.84      0.86      0.85      1605
              Мир       0.76      0.83      0.79      4079
  Наука и техника       0.81      0.81      0.81      1579
      Путешествия       0.63      0.47      0.54       192
           Россия       0.74      0.81      0.77      4766
Силовые структуры       0.48      0.22      0.30       585
            Спорт       0.96      0.97      0.96      1926
         Ценности       0.84      0.74      0.79       

In [39]:
preds = logreg.predict(X_test_W2V)
print(classification_report(test_df['topic'], preds))

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                   precision    recall  f1-score   support

   69-я параллель       0.33      0.03      0.05        38
           Бизнес       0.36      0.07      0.12       216
      Бывший СССР       0.78      0.75      0.76      1557
              Дом       0.82      0.73      0.77       646
         Из жизни       0.63      0.52      0.57       828
   Интернет и СМИ       0.71      0.65      0.68      1326
             Крым       0.00      0.00      0.00        20
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.84      0.86      0.85      1605
              Мир       0.76      0.83      0.79      4079
  Наука и техника       0.80      0.81      0.80      1579
      Путешествия       0.69      0.47      0.56       192
           Россия       0.74      0.81      0.77      4766
Силовые структуры       0.50      0.24      0.32       585
            Спорт       0.96      0.97      0.96      1926
         Ценности       0.84      0.72      0.77       

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [40]:
preds = logregNavec.predict(X_test_navec)
print(classification_report(test_df['topic'], preds))

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                   precision    recall  f1-score   support

   69-я параллель       0.12      0.03      0.04        38
           Бизнес       0.44      0.10      0.16       216
      Бывший СССР       0.79      0.79      0.79      1557
              Дом       0.81      0.73      0.77       646
         Из жизни       0.62      0.54      0.57       828
   Интернет и СМИ       0.72      0.67      0.69      1326
             Крым       0.33      0.05      0.09        20
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.84      0.86      0.85      1605
              Мир       0.77      0.82      0.79      4079
  Наука и техника       0.79      0.82      0.80      1579
      Путешествия       0.64      0.48      0.55       192
           Россия       0.75      0.80      0.77      4766
Силовые структуры       0.53      0.28      0.36       585
            Спорт       0.95      0.98      0.96      1926
         Ценности       0.80      0.70      0.75       

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [41]:
preds = logregRus.predict(X_test_rv)
print(classification_report(test_df['topic'], preds))

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                   precision    recall  f1-score   support

   69-я параллель       0.00      0.00      0.00        38
           Бизнес       0.00      0.00      0.00       216
      Бывший СССР       0.00      0.00      0.00      1557
              Дом       0.00      0.00      0.00       646
         Из жизни       0.00      0.00      0.00       828
   Интернет и СМИ       0.00      0.00      0.00      1326
             Крым       0.00      0.00      0.00        20
    Культпросвет        0.00      0.00      0.00        10
         Культура       0.00      0.00      0.00      1605
              Мир       0.00      0.00      0.00      4079
  Наука и техника       0.00      0.00      0.00      1579
      Путешествия       0.00      0.00      0.00       192
           Россия       0.22      1.00      0.36      4766
Силовые структуры       0.00      0.00      0.00       585
            Спорт       0.00      0.00      0.00      1926
         Ценности       0.00      0.00      0.00       

C:\Users\lsd24\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Как можно видеть, эмбединги rusvectors дают совсем плохое качество, а остальные решение, такие как обученный нами w2v, navec и комбинированнная модель w2v и tf-idf дают неплохое и одинаковое качество